# Analyse a picture for project move color prototype

## Import and variables

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
DIST = 2
BORDER_OF_PREDICTION = 0.5

In [ ]:
PLT_RED = 'r+'
PLT_BLUE = 'bo'
PLT_GREEN = 'g.'

## Functions

In [ ]:
def prediction(combined_squared_error):
    return (1 / (1 + combined_squared_error))

In [ ]:
def error_function(hsv_pixel):
    # 1/ ( 1 + combined_error^2)
    # hsv_pixel = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)

    error_hue = hue - hsv_pixel[0, 0, 0]
    error_sat = sat - hsv_pixel[0, 0, 1]
    error_val = val - hsv_pixel[0, 0, 2]

    combined_squared_error = (
        error_hue**2 * factor_hue
        + error_sat**2 * factor_sat 
        + error_val**2 * factor_val
    )
    
    predicted_value = prediction(combined_squared_error)
    return combined_squared_error, predicted_value

In [ ]:
def draw_pixel_value(event, x, y, flags, param):
    global clicked
    global marked_good_pixels
    global marked_bad_pixels

    # if event != 0:
    #     print(f'event:{event}')

    if event == cv2.EVENT_LBUTTONDOWN and not clicked:
        clicked = True
        pixel = image[y, x]
        pixel = np.uint8([[pixel]])
        hsv_pixel = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
        
        # print(f'draw_pixel_value/pixel:{pixel}')
        # print(f'draw_pixel_value/hsv_pixel:{hsv_pixel}')
        squared_error_and_prediction = error_function(hsv_pixel)
        #print(predicted_value)
        draw_function((x,y), squared_error_and_prediction, hsv_pixel)
        marked_good_pixels.append(hsv_pixel[0, 0, :])
        
    elif event == cv2.EVENT_RBUTTONDOWN and not clicked:
        clicked = True
        pixel = image[y, x]
        pixel = np.uint8([[pixel]])
        hsv_pixel = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
        
        # print(f'draw_pixel_value/pixel:{pixel}')
        # print(f'draw_pixel_value/hsv_pixel:{hsv_pixel}')
        squared_error_and_prediction = error_function(hsv_pixel)
        #print(predicted_value)
        draw_function((x,y), squared_error_and_prediction, hsv_pixel)
        marked_bad_pixels.append(hsv_pixel[0 , 0, :])
        
    elif event == cv2.EVENT_LBUTTONUP or cv2.EVENT_RBUTTONUP:
        clicked = False

In [ ]:
def draw_function(point_in_image, calculated_point, hsv_pixel):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    circle_size = rgb_image.shape[0] * 10 // 640
    text_size = rgb_image.shape[0] * 4 // 640

    cv2.circle(rgb_image, point_in_image, 2*circle_size, [0,0,0], thickness=-1)
    cv2.circle(rgb_image, point_in_image, circle_size, [255,255,255], thickness=-1)
    cv2.putText(rgb_image, f'{point_in_image}', (10, rgb_image.shape[0]-10), cv2.FONT_HERSHEY_SIMPLEX, text_size, (255,255,255))
    
    x_min = np.min([calculated_point[0]-2, -10])
    x_max = np.max([calculated_point[0]+2, 10])
    x_step = np.round((x_max - x_min) / 100, decimals=2)
    #x_func = np.arange(0, 10, 0.05)
    x_func = np.arange(x_min, x_max, x_step)
    y_func = list(map(prediction, x_func**2))
    #print(x_func)
    #print(y_func)

    fig, axes = plt.subplot_mosaic([
        ['image', 'graph'],
        ['pixel', 'graph']
    ])
    
    fig.set_dpi(200)
    #fig.set_size_inches(10,10)
    
    axes['image'].imshow(rgb_image)
    
    axes['graph'].plot(x_func, y_func)
    axes['graph'].plot([x_min, calculated_point[0], calculated_point[0]], [calculated_point[1], calculated_point[1], 0], 'r--')
    axes['graph'].plot(calculated_point[0], calculated_point[1], 'ro')

    pixel = cv2.cvtColor(hsv_pixel, cv2.COLOR_HSV2RGB)
    pixel_example = np.zeros((100, 100, 3), dtype=np.uint8)
    pixel_example[:, :, 0] = pixel[0, 0, 0]
    pixel_example[:, :, 1] = pixel[0, 0, 1]
    pixel_example[:, :, 2] = pixel[0, 0, 2]

    axes['pixel'].imshow(pixel_example)
    
    #plt.figtext(.75, .8, f'result: {calculated_point[1]}')
    fig.suptitle(f'HSV:{hsv_pixel[0, 0, :]} result: {calculated_point[1]:.5f}')
    plt.show()
    
    #print(calculated_point)

In [ ]:
def click_and_evaluate_pixel(image):
    title = 'click at pixel'

    cv2.namedWindow(title)
    cv2.setMouseCallback(title, draw_pixel_value)
    
    while True:
        cv2.imshow(title, image)
        if cv2.waitKey(30) == 27:
            break
    
    cv2.destroyAllWindows()

## Load images and settings

In [ ]:
image_filename = 'debug_image.jpg'

In [ ]:
image = cv2.imread(image_filename)
plt.imshow(image)

In [ ]:
settings_filename = 'debug_hsv.txt'
settings = [''] * 6

with open(settings_filename, 'r') as fid:
    for i in range(len(settings)):
        settings[i] = fid.readline().strip()

In [ ]:
hue = float(settings[0])
sat = float(settings[1])
val = float(settings[2])
factor_hue = float(settings[3])
factor_sat = float(settings[4])
factor_val = float(settings[5])

In [ ]:
print(hue)
print(sat)
print(val)
print(factor_hue)
print(factor_sat)
print(factor_val)

## Image validation

In [ ]:
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_image)

In [ ]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [ ]:
for col in range(0, hsv_image.shape[0], DIST):
    for row in range(0, hsv_image.shape[1], DIST):
        hsv_pixel = np.uint8([[hsv_image[col, row, :]]])
        #print(pixel)
        squared_error_and_prediction = error_function(hsv_pixel)
        if squared_error_and_prediction[1] >= BORDER_OF_PREDICTION:
            rgb_image[col, row, :] = (0, 0, 255)
        else:
            rgb_image[col, row, :] = (255, 0, 0)

In [ ]:
plt.imshow(rgb_image)

In [ ]:
colored_bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

In [ ]:
cv2.imshow('calibration result', colored_bgr_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

## Histograms and plots

In [ ]:
def get_roi(image, description=''):
    x, y, w, h = cv2.selectROI(description, image, False, False)
    cv2.destroyAllWindows()
    
    print(x, y, w, h)
    
    x_lower = x
    x_upper = x + w
    y_lower = y
    y_upper = y + h
    
    roi = image[y_lower:y_upper, x_lower:x_upper]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    plt.imshow(roi)
    return x_lower, x_upper, y_lower, y_upper

In [ ]:
x_lower, x_upper, y_lower, y_upper = get_roi(image, 'only positives')

In [ ]:
neg_x_lower, neg_x_upper, neg_y_lower, neg_y_upper = get_roi(image, 'all positives')

In [ ]:
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [ ]:
good_values = []
bad_values = []
for col in range(0, hsv_image.shape[0], DIST):
    for row in range(0, hsv_image.shape[1], DIST):
        if y_lower < col < y_upper:
            if x_lower < row < x_upper:
                good_values.append(hsv_image[col, row, :])
        if neg_y_lower > col or col > neg_y_upper:
            bad_values.append(hsv_image[col, row, :])
        elif neg_x_lower > row or row > neg_x_upper:
            bad_values.append(hsv_image[col, row, :])

In [ ]:
good_values = np.array(good_values)

In [ ]:
good_values.shape

In [ ]:
bad_values = np.array(bad_values)

In [ ]:
bad_values.shape

In [ ]:
good_hue = good_values[:, 0]
good_sat = good_values[:, 1]
good_val = good_values[:, 2]

bad_hue = bad_values[:, 0]
bad_sat = bad_values[:, 1]
bad_val = bad_values[:, 2]

fig, axes = plt.subplots(1, 3)
fig.suptitle('good pixels')
_ = axes[0].hist(good_hue, bins=36, range=(0, 180))
_ = axes[0].set_xlabel('hue')
_ = axes[1].hist(good_sat, bins=51, range=(0, 255))
_ = axes[1].set_xlabel('sat')
_ = axes[2].hist(good_val, bins=51, range=(0, 255))
_ = axes[2].set_xlabel('val')

fig, axes = plt.subplots(1, 3)
fig.suptitle('bad values')
_ = axes[0].hist(bad_hue, bins=36, range=(0, 180), color='red')
_ = axes[0].set_xlabel('hue')
_ = axes[1].hist(bad_sat, bins=51, range=(0, 255), color='red')
_ = axes[1].set_xlabel('sat')
_ = axes[2].hist(bad_val, bins=51, range=(0, 255), color='red')
_ = axes[2].set_xlabel('val')

In [ ]:
nbr_entries_one_direction = 400
range_size = nbr_entries_one_direction * nbr_entries_one_direction
pos_hue_range = np.zeros((range_size))
pos_sat_range = np.zeros((range_size))
pos_val_range = np.zeros((range_size))

radius = 0.5
err2 = (1 - radius) / radius

In [ ]:
offset = 10

min_zoom_plot_hue = min(good_hue) - offset
max_zoom_plot_hue = max(good_hue) + offset
min_zoom_plot_sat = min(good_sat) - offset
max_zoom_plot_sat = max(good_sat) + offset
min_zoom_plot_val = min(good_val) - offset
max_zoom_plot_val = max(good_val) + offset

In [ ]:
fig = plt.figure(layout='constrained')
fig.suptitle('position comparison')
axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')

fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((min_zoom_plot_val, max_zoom_plot_val))

fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison without positives')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((min_zoom_plot_val, max_zoom_plot_val))

## Click and evaluate

In [ ]:
settings

In [ ]:
# optional
hue = float(settings[0])
sat = float(settings[1])
val = float(settings[2])
factor_hue = float(0)
factor_sat = float(0)
factor_val = float(settings[5])

In [ ]:
clicked = False
marked_good_pixels = []
marked_bad_pixels = []

In [ ]:
# left click good, right click bad
click_and_evaluate_pixel(colored_bgr_image)

In [ ]:
marked_good_pixels = np.array(marked_good_pixels)
marked_bad_pixels = np.array(marked_bad_pixels)

In [ ]:
marked_good_pixels.shape

In [ ]:
marked_bad_pixels.shape

In [ ]:
single_example_size = 200

In [ ]:
if marked_good_pixels.size > 0:
    good_color_example = np.ones((single_example_size, len(marked_good_pixels)*single_example_size, 3), dtype=np.uint8)
    
    for pos in range(len(marked_good_pixels)):
        good_color_example[:, pos * single_example_size : (pos+1) * single_example_size, :] = good_color_example[0, pos * single_example_size, :] * marked_good_pixels[pos]
        
        if pos != 0:
            good_color_example[:, pos * single_example_size, :] = 0
        
    good_color_example = cv2.cvtColor(good_color_example, cv2.COLOR_HSV2RGB)
    
    for pos in range(len(marked_good_pixels)):
        prediction_ = error_function(np.uint8([[good_color_example[0, pos*single_example_size, :]]]))[1]
        cv2.putText(good_color_example, f'{prediction_:0.4f}', (pos*single_example_size, good_color_example.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
    
    plt.title('good colors')
    plt.imshow(good_color_example)
    plt.show()

if marked_bad_pixels.size > 0:

    bad_color_example = np.ones((single_example_size, len(marked_bad_pixels)*single_example_size, 3), dtype=np.uint8)
    
    for pos in range(len(marked_bad_pixels)):
        bad_color_example[:, pos * single_example_size : (pos+1) * single_example_size, :] = bad_color_example[0, pos * single_example_size, :] * marked_bad_pixels[pos]
        if pos != 0:
            bad_color_example[:, pos * single_example_size, :] = 0
        
    bad_color_example = cv2.cvtColor(bad_color_example, cv2.COLOR_HSV2RGB)

    for pos in range(len(marked_bad_pixels)):
        predicton_ = error_function(np.uint8([[bad_color_example[0, pos*single_example_size, :]]]))[1]
        cv2.putText(bad_color_example, f'{prediction_:0.4f}', (pos*single_example_size, bad_color_example.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    plt.title('bad colors')
    plt.imshow(bad_color_example)
    plt.show()

In [ ]:
good_hue = marked_good_pixels[:, 0]
good_sat = marked_good_pixels[:, 1]
good_val = marked_good_pixels[:, 2]

bad_hue = marked_bad_pixels[:, 0]
bad_sat = marked_bad_pixels[:, 1]
bad_val = marked_bad_pixels[:, 2]

fig, axes = plt.subplots(1, 3)
fig.suptitle('good pixels')
_ = axes[0].hist(good_hue, bins=36, range=(0, 180))
_ = axes[0].set_xlabel('hue')
_ = axes[1].hist(good_sat, bins=51, range=(0, 255))
_ = axes[1].set_xlabel('sat')
_ = axes[2].hist(good_val, bins=51, range=(0, 255))
_ = axes[2].set_xlabel('val')

fig, axes = plt.subplots(1, 3)
fig.suptitle('bad values')
_ = axes[0].hist(bad_hue, bins=36, range=(0, 180), color='red')
_ = axes[0].set_xlabel('hue')
_ = axes[1].hist(bad_sat, bins=51, range=(0, 255), color='red')
_ = axes[1].set_xlabel('sat')
_ = axes[2].hist(bad_val, bins=51, range=(0, 255), color='red')
_ = axes[2].set_xlabel('val')

offset = 10

min_zoom_plot_hue = min(good_hue) - offset
max_zoom_plot_hue = max(good_hue) + offset
min_zoom_plot_sat = min(good_sat) - offset
max_zoom_plot_sat = max(good_sat) + offset
min_zoom_plot_val = min(good_val) - offset
max_zoom_plot_val = max(good_val) + offset

fig = plt.figure(layout='constrained')
fig.suptitle('position comparison')
axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')PLT_GREEN
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')

fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((min_zoom_plot_val, max_zoom_plot_val))

fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison without positives')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((min_zoom_plot_val, max_zoom_plot_val))